# Visualize the time series of 30-m fused images from Landsat and MODIS

## A contract project of Asia Development Bank

### Contractor:
* Kaiyu Guan
* Zhan Li

In [1]:
import os
import sys
import itertools
import glob
import warnings
from collections import OrderedDict

import numpy as np
import pandas as pd
from osgeo import gdal, gdal_array, osr, ogr

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
sns.set_context("paper")
sns.set_style("whitegrid")
dpi = 300

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook
import plotly.tools

In [6]:
# time series file by fusion of Landsat and MODIS BEFORE TIMESAT smoothing
ts_ndvi_fused_file = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-fusion-ts/predicted_NDVI"

# time series file by fusion of Landsat and MODIS AFTER TIMESAT smoothing
ts_ndvi_timesat_file = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-fusion-ts/fit_NDVI"

In [6]:
# # the lat and lon of points of which time series to be visualized
# geo_points = { \
#               "Point 1":(106.378046, 20.429023), \
#               "Point 2":(106.506207, 20.461707), \
#               "Point 3":(106.46351961, 20.48002966), \
#               "Point 4":(106.46235387, 20.47841337) \
#              }
# geo_points = { \
#               "SGveg-LSATcrop": (106.07962212, 20.61375514), \
#               "SGveg-LSATbarren": (106.08106149, 20.61374614), \
#               "SGcrop-LSATcrop": (106.31273800, 20.57257159)}

# # convert the geographic coordinates of given points to image coordinates
# img_points = {pk:geo2Pixel(ts_ndvi_fused_file, geo_points[pk][0], geo_points[pk][1]) for pk in geo_points.keys()}
# proj_points = {pk:geo2Proj(ts_ndvi_fused_file, geo_points[pk][0], geo_points[pk][1]) for pk in geo_points.keys()}

In [7]:
# # randomly select several points to visualize
# npts = 4
# ts_ndvi_fused_meta = get_raster_meta_GDAL(ts_ndvi_fused_file)
# img_points = {"Point {0:d}".format(i+1):(np.random.randint(0, ts_ndvi_fused_meta["RasterXSize"]), \
#                                        np.random.randint(0, ts_ndvi_fused_meta["RasterYSize"])) for i in range(npts)}

# # the sample (column) and line (row) of the points in the image
# img_points = { \
#               "Point 1":(1094, 1192), \
#               "Point 2":(1460, 1475) \
#              }

# # Convert image coordinates to geographic and projected coordinates
# geo_points = {k:pixel2Geo(ts_ndvi_fused_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
# proj_points = {k:pixel2Proj(get_raster_meta_GDAL(ts_ndvi_fused_file)["GeoTransform"], img_points[k][0], img_points[k][1]) for k in img_points.keys()}

## Select points from specific classes from the classification map

In [7]:
# Give classification image files
cls_img_file_list = [\
                     "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_lsat_scenes.img", \
                     "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_sg_ndvi.img"]

cls_img_label_list = [\
                      "SR_3_Scenes", \
                      "NDVI_TS_SG"]

cls_rnd_sample_shapefile = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_lsat_scenes_rnd_samples.shp"

# read classification map
from classify_image import ImageClassifier
ic = ImageClassifier()
cls_map_profile_list = [ic.getRasterProfile(cif) for cif in cls_img_file_list]

ncls_list = [int(cls_map_profile['Metadata']['ENVI']['classes']) - 1 for cls_map_profile in cls_map_profile_list]
no_data = 0
cls_map_list = [ic.readRaster(cif)[0][0] for cif in cls_img_file_list]
cls_map_masked_list = [np.ma.masked_equal(cm, no_data) for cm in cls_map_list]

# Designate the map to use
n = 0
cls_map = cls_map_list[n]
cls_map_profile = cls_map_profile_list[n]
no_data = 0

cls_code = np.arange(int(cls_map_profile['Metadata']['ENVI']['classes'])-1, dtype=np.int8)+1
cls_name = cls_map_profile['Metadata']['ENVI']['class_names'].strip('{}').split(',')[1:]
cls_name = [cn.strip() for cn in cls_name]
cls_npix = np.array([np.sum(cls_map==cls) for cls in cls_code])
total_npix = np.sum(cls_npix)
cls_w = cls_npix / float(total_npix)
print cls_code
print cls_name
print cls_npix
print cls_w

[1 2 3 4 5 6]
['croplands', 'barren', 'urban_and_built_up', 'water_bodies', 'wetlands', 'natural_terrestrial_veg']
[1901126  105711 1204497  729353   47654    4617]
[ 0.47611971  0.02647436  0.30165531  0.18265982  0.01193451  0.00115629]


In [8]:
# Read reference data
driver = ogr.GetDriverByName("ESRI Shapefile")
data_src = driver.Open(cls_rnd_sample_shapefile, 0)
layer = data_src.GetLayer()

npts = layer.GetFeatureCount()
layer_def = layer.GetLayerDefn()
nfields = layer_def.GetFieldCount()
# get name of fields
field_names = [layer_def.GetFieldDefn(i).GetName() for i in range(nfields)]
# set up a pandas dataframe to read the feature attributes
cls_ref_df = pd.DataFrame(np.zeros((npts, nfields)), columns=field_names)

for i, feature in enumerate(layer):
    tmp_list = [feature.GetField(fd) for fd in field_names]
    cls_ref_df.iloc[i, :] = np.array([0 if item is None else np.float(item) for item in tmp_list])

data_src.Destroy()

# Fill the secondary reference labels
tmp_flag = cls_ref_df['SecRefCode'] == 0
cls_ref_df.loc[tmp_flag, 'SecRefCode'] = cls_ref_df.loc[tmp_flag, 'PriRefCode']

# Get the classification labels
cls_ref_df['ClsCode'] = cls_map[cls_ref_df['Line'].astype(int), cls_ref_df['Sample'].astype(int)]

In [9]:
img_points = OrderedDict()

# Randomly select several pixels with correct cropland classification and 
# several with wrong cropland classification
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']==1, cls_ref_df['PriRefCode']==1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']==1, cls_ref_df['PriRefCode']!=1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']!=1, cls_ref_df['PriRefCode']==1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(reduce(np.logical_and, (cls_ref_df['ClsCode']!=1, cls_ref_df['PriRefCode']!=1, cls_ref_df['ClsCode']==cls_ref_df['PriRefCode'])))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))

In [10]:
# Convert image coordinates to geographic and projected coordinates
geo_points = OrderedDict([(k, pixel2Geo(ts_ndvi_fused_file, img_points[k][0]+0.5, img_points[k][1]+0.5)) for k in img_points.keys()])
proj_points = OrderedDict([(k, pixel2Proj(get_raster_meta_GDAL(ts_ndvi_fused_file)["GeoTransform"], img_points[k][0]+0.5, img_points[k][1]+0.5)) for k in img_points.keys()])

## Time series from stacked STARFM+Landsat and TIMESAT DL fitting

In [12]:
ts_ndvi_fused_data = { k:read_pixel_GDAL(ts_ndvi_fused_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
ts_ndvi_fused_doy = np.arange(len(ts_ndvi_fused_data[ts_ndvi_fused_data.keys()[0]]))+1
# sort the data points according to doy
sort_ind = np.argsort(ts_ndvi_fused_doy)
ts_ndvi_fused_data = {k:ts_ndvi_fused_data[k][sort_ind] for k in ts_ndvi_fused_data.keys()}
ts_ndvi_fused_doy = ts_ndvi_fused_doy[sort_ind]

In [13]:
ts_ndvi_timesat_data = { k:read_pixel_GDAL(ts_ndvi_timesat_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
ts_ndvi_timesat_doy = np.arange(len(ts_ndvi_timesat_data[ts_ndvi_timesat_data.keys()[0]]))+1
# sort the data points according to doy
sort_ind = np.argsort(ts_ndvi_timesat_doy)
ts_ndvi_timesat_data = {k:ts_ndvi_timesat_data[k][sort_ind] for k in ts_ndvi_timesat_data.keys()}
ts_ndvi_timesat_doy = ts_ndvi_timesat_doy[sort_ind]

In [14]:
ts_ndvi_fused_meta = get_raster_meta_GDAL(ts_ndvi_fused_file)
# get the nodata value of the given image
if ts_ndvi_fused_meta["NoDataValue"] is None:
    nodata = -9999

* Selected DOY for image visualization
    * Around second peak: 242, 245
    * Around the trough: 186, 192
    * Last day in the year: 365, 365

## Time series from STARFM fusion

In [15]:
# red_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.red.bin")
# nir_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.nir.bin")
# ndvi_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.ndvi.bin")

# fnames, red_ts_data = get_ts_from_imgs(red_imgfiles, img_points)
# red_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(red_doy)
# red_ts_data = {k:red_ts_data[k][sort_ind] for k in red_ts_data.keys()}
# red_doy = red_doy[sort_ind]

# fnames, nir_ts_data = get_ts_from_imgs(nir_imgfiles, img_points)
# nir_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(nir_doy)
# nir_ts_data = {k:nir_ts_data[k][sort_ind] for k in nir_ts_data.keys()}
# nir_doy = nir_doy[sort_ind]

# fnames, ndvi_ts_data = get_ts_from_imgs(ndvi_imgfiles, img_points)
# ndvi_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(ndvi_doy)
# ndvi_ts_data = {k:ndvi_ts_data[k][sort_ind] for k in ndvi_ts_data.keys()}
# ndvi_doy = ndvi_doy[sort_ind]

In [16]:
markers = mpl.markers.MarkerStyle().markers

In [17]:
# plot_ts([red_doy, nir_doy], [red_ts_data, nir_ts_data], geo_points, \
#         style_list=['.r', '.b'], \
#         plot_kw_dict_list=[dict(label="Red"), dict(label="NIR")], \
#         use_plotly=True)

In [18]:
# plot_ts([np.arange(len(ts_ndvi_fused_data[ts_ndvi_fused_data.keys()[0]]))+1, ndvi_doy], [ts_ndvi_fused_data, ndvi_ts_data], geo_points, \
#         style_list=['.r', '.g'], \
#         plot_kw_dict_list=[dict(label="NDVI Predicted"), dict(label="NDVI STARFM")], \
#         use_plotly=True)

## Time series from TIMESAT SG fitting to STARFM+Landsat

In [19]:
fitsg_ndvi_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-ts-sg/fitSG_NDVI_126046.2015[0-9][0-9][0-9]")

fnames, sg_ndvi_ts_data = get_ts_from_imgs(fitsg_ndvi_imgfiles, img_points)
sg_ndvi_doy = np.array([int(imgf.split(".")[1][4:]) for imgf in fnames])
# sort the data points according to doy
sort_ind = np.argsort(sg_ndvi_doy)
sg_ndvi_ts_data = {k:sg_ndvi_ts_data[k][sort_ind] for k in sg_ndvi_ts_data.keys()}
sg_ndvi_doy = sg_ndvi_doy[sort_ind]

In [ ]:
plot_ts([ts_ndvi_fused_doy, ts_ndvi_timesat_doy, sg_ndvi_doy], \
        [ts_ndvi_fused_data, ts_ndvi_timesat_data, sg_ndvi_ts_data], geo_points, \
        style_list=['.r', '-b', '-k'], \
        plot_kw_dict_list=[dict(label="NDVI Predicted (fused + LC8)"), dict(label="NDVI TIMESAT Double Logistic"), dict(label="NDVI TIMESAT SG")], \
        use_plotly=True)